### Imputation of Missing Data and Financial Analysis

#### Packages

In [1]:
# Load packages and check versions
import sys
import sklearn
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt

%matplotlib inline


print('The Python version is {}.\n'.format(sys.version))
print('The Numpy version is {}.\n'.format(np.__version__))
print('The Pandas version is {}.\n'.format(pd.__version__))
print('The Matplotlib version is {}.\n'.format(mpl.__version__))
print('The Scikit-Learn version is {}.\n'.format(sklearn.__version__))

The Python version is 3.8.5 (default, Jul 28 2020, 12:59:40) 
[GCC 9.3.0].

The Numpy version is 1.19.1.

The Pandas version is 1.1.0.

The Matplotlib version is 3.3.3.

The Scikit-Learn version is 0.24.0.



### Imputation of missing data

Imputation involves using the known values of a certain feature to make a best educated guess as to the missing values as a way to solve the problem of missing data that might have potential predictive power. 

Following are some strategies to deal with missing data:

- Ignore samples with missing data, or features with missing data
- Impute missing feature values with a constant (e.g. mean, median or mode) for a specific category
- Impute missing values using a random selection from non-missing values
- Impute missing values using interpolation of nearby observations in time or space
- Mode-based imputation

While the first option doesn't actually impute any data, it is a common practice among data science projects when dealing with missing values. 

The simplest types of imputation involve using a summary statistic of the non-missing feature values, as the single constant value with which to replace all the missing values. This summary statistic may be the mean, median, or mode for continuous features. For categorial features, the mode is an option, as well as the median for ordinal categoricals. An important additional case of imputation with a constant value for categorial variables is to create a new level of the categorical  variable, to indicate data is missing.

More sophisticated methods of imputation can be used to fill in missing values with non-constant values. These methods reflect an acknowledgment that the missing values may not all be equal. he simplest way to do this is to fill in missing values using a random draw from the set of non-missing values, with replacement. This way, the relative frequency of different values chosen to fill in missing data will be similar to the existing data for that feature.

In case samples are located within time or space, such as a time series or geolocated data, then temporal or spatial interpolation methods may be used. These methods follow the general idea that a missing data point is probably located somewhere between the values of adjacent data points in time or space.

Lastly, perhaps the most sophisticated way to fill in missing data is to view the imputation problem as a predictive modeling "problem within a problem." In this method, the feature with missing values is considered the response variable, while the features with no missing values are considered the features of this sub-problem.

### Preparing Samples with Missing Data

Note that for the case study dataset, the feature that had missing values was **pay_1**, which turned out to be the most important feature as dentified by both the univariate feature selection as well as the feature importance of the random forest. In order to test out different imputation strategies on the case study data and see how they affect the predictive capability of the modeling approach, it is necesasry to replace samples with missing data back in to the dataset.

Let us load the original dataset:

In [2]:
df = pd.read_excel('data/default_of_credit_card_clients__courseware_version_1_21_19.xls')
df.columns = [col.lower() for col in df.columns]
df.head()

,id,limit_bal,sex,education,marriage,age,pay_1,pay_2,pay_3,pay_4,...,bill_amt4,bill_amt5,bill_amt6,pay_amt1,pay_amt2,pay_amt3,pay_amt4,pay_amt5,pay_amt6,default payment next month
0,798fc410-45c1,20000,2,2,1,24,2,2,-1,-1,...,0,0,0,0,689,0,0,0,0,1
1,8a8c8f3b-8eb4,120000,2,2,2,26,-1,2,0,0,...,3272,3455,3261,0,1000,1000,1000,0,2000,1
2,85698822-43f5,90000,2,2,2,34,0,0,0,0,...,14331,14948,15549,1518,1500,1000,1000,1000,5000,0
3,0737c11b-be42,50000,2,2,1,37,0,0,0,0,...,28314,28959,29547,2000,2019,1200,1100,1069,1000,0
4,3b7f77cc-dbc0,50000,1,2,1,57,-1,0,-1,0,...,20940,19146,19131,2000,36681,10000,9000,689,679,0


Looking at the zero rows:

In [3]:
df_zero_mask = df == 0
features_zero_mask = df_zero_mask.iloc[:, 1:].all(axis=1)

(sum(features_zero_mask),
 features_zero_mask)

(315,
 0        False
 1        False
 2        False
 3        False
 4        False
          ...  
 29995    False
 29996    False
 29997    False
 29998    False
 29999    False
 Length: 30000, dtype: bool)

Let us remove them from the dataframe and check whether all rows are unique:

In [4]:
df_clean = df.loc[~features_zero_mask].copy()

df_clean.shape

(29685, 25)

In [5]:
df_clean.id.nunique()

29685

Let us replace the undocumented values for the **education** and **marriage** columns with an *unknown* value.

Education shoul be "Education (1 = graduate school; 2 = university; 3 = high school; 4 = others)"

Marriage should be "Marriage (1 = married; 2 = single; 3 = others)"

In [6]:
(df_clean.education.value_counts(), 
 df_clean.marriage.value_counts())

(2    13884
 1    10474
 3     4867
 5      275
 4      122
 6       49
 0       14
 Name: education, dtype: int64,
 2    15810
 1    13503
 3      318
 0       54
 Name: marriage, dtype: int64)

In [7]:
df_clean.education.replace(to_replace=[0,5,6], value=4, inplace=True)
df_clean.marriage.replace(to_replace=0, value=3, inplace=True)

Let us maintain the **pay_1** with missing values for the imputation process. However, let us "save" them into another set to include it the train and test set in a way to compare whether the imputation process reflects a better performance:

In [8]:
df_clean.pay_1.value_counts()

0                13087
-1                5047
1                 3261
Not available     3021
-2                2476
2                 2378
3                  292
4                   63
5                   23
8                   17
6                   11
7                    9
Name: pay_1, dtype: int64

In [11]:
missing_pay_1_mask = df_clean.pay_1 == 'Not available'
sum(missing_pay_1_mask)

3021

Now let us isolate these values:

In [15]:
df_missing_pay_1 = df_clean[missing_pay_1_mask].copy()
df_missing_pay_1

,id,limit_bal,sex,education,marriage,age,pay_1,pay_2,pay_3,pay_4,...,bill_amt4,bill_amt5,bill_amt6,pay_amt1,pay_amt2,pay_amt3,pay_amt4,pay_amt5,pay_amt6,default payment next month
17,f0ba5e02-0c0f,320000,1,1,1,49,Not available,0,0,-1,...,70074,5856,195599,10358,10000,75940,20000,195599,50000,0
28,aecb449b-a999,50000,2,3,1,47,Not available,-1,-1,-1,...,2040,30430,257,3415,3421,2044,30430,257,0,0
29,57ea9a1f-cdd5,50000,1,1,2,26,Not available,0,0,0,...,17907,18375,11400,1500,1500,1000,1000,1600,0,0
54,781f3cc2-7ddc,150000,2,1,2,29,Not available,0,0,0,...,26518,21042,16540,1600,1718,1049,1500,2000,5000,0
60,5c00d7e6-a804,500000,2,3,1,28,Not available,0,0,0,...,14937,13827,15571,1516,1300,1000,1000,2000,2000,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29949,4b490335-91ab,280000,1,1,1,39,Not available,0,0,0,...,40939,44290,70573,1652,2144,1666,5015,32013,12,0
29956,b4a99d86-3bdd,150000,1,1,2,30,Not available,0,0,-1,...,1610,1180,2980,2000,2000,2000,1000,3000,0,0
29967,f411fbb3-adaf,50000,1,2,1,32,Not available,0,0,-1,...,50551,40590,39384,2686,0,50551,0,1600,1500,0
29973,31c0c152-e5ac,230000,1,2,1,35,Not available,-2,-2,-2,...,0,0,0,0,0,0,0,0,0,1


Let us now load the previously cleaned dataframe:

In [16]:
df_cleaned_before = pd.read_csv('data/default_of_credit_card_clients_cleaned.csv')
df_cleaned_before.head()

,id,limit_bal,sex,education,marriage,age,pay_1,pay_2,pay_3,pay_4,...,pay_amt3,pay_amt4,pay_amt5,pay_amt6,default_payment_next_month,education_cat,graduate_school,high_school,others,university
0,798fc410-45c1,20000,2,2,1,24,2,2,-1,-1,...,0,0,0,0,1,university,0,0,0,1
1,8a8c8f3b-8eb4,120000,2,2,2,26,-1,2,0,0,...,1000,1000,0,2000,1,university,0,0,0,1
2,85698822-43f5,90000,2,2,2,34,0,0,0,0,...,1000,1000,1000,5000,0,university,0,0,0,1
3,0737c11b-be42,50000,2,2,1,37,0,0,0,0,...,1200,1100,1069,1000,0,university,0,0,0,1
4,3b7f77cc-dbc0,50000,1,2,1,57,-1,0,-1,0,...,10000,9000,689,679,0,university,0,0,0,1


Let us isolate the independent features:

In [18]:
features_response = df_cleaned_before.columns.tolist()
to_remove = ['id', 'sex'] + ['pay_{}'.format(i) for i in range(2,7)] + features_response[-6:]

features_response = [feature for feature in features_response if feature not in to_remove]
features_response

['limit_bal',
 'education',
 'marriage',
 'age',
 'pay_1',
 'bill_amt1',
 'bill_amt2',
 'bill_amt3',
 'bill_amt4',
 'bill_amt5',
 'bill_amt6',
 'pay_amt1',
 'pay_amt2',
 'pay_amt3',
 'pay_amt4',
 'pay_amt5',
 'pay_amt6']

Now it is possible to test different imputation techniques for the *df_missing_pay_1*.

### Mode and Random Imputation of pay_1